In [1]:
import pandas as pd
import json


df = pd.read_excel("validate.xlsx")
df = df.head(5)
print(df.head())

import analysisrag
import anaylysis

for i in range(len(df)):
    code = df.iloc[i]["code"]
    try: 
        vulnerabilities = analysisrag.get_vulnerabilities(code)
        df.at[i, "vulnerabilities"] = json.dumps(vulnerabilities)
    except Exception as e:
        print("error", e)
        df.at[i, "vulnerabilities"] = "nan"
    print("vulnerabilities!!!!", vulnerabilities)


                                                code  \
0  // SPDX-License-Identifier: MIT\npragma solidi...   
1  // SPDX-License-Identifier: MIT\npragma solidi...   
2  // SPDX-License-Identifier: MIT\npragma solidi...   
3  // SPDX-License-Identifier: MIT\npragma solidi...   
4  // SPDX-License-Identifier: MIT\npragma solidi...   

                                             contain  \
0  [{\n    "line": "13",\n    "token": "payable(m...   
1                                                 []   
2  [{\n    "line": "8",\n    "token": "balances[m...   
3  [\n  {\n    "line": "14",\n    "token": "Data ...   
4  [\n  {\n    "line": "10",\n    "token": "tx.or...   

                                                miss  
0                                                 []  
1  [{\n    "line": "12",\n    "token": "payable(m...  
2                                                 []  
3                                                 []  
4                                                 [

In [2]:
from nltk.translate.bleu_score import sentence_bleu

def compare_strings(str1, str2):
    """
    Compare two strings using BLEU score.
    """
    reference = [str1.split()]
    candidate = str2.split()
    score = sentence_bleu(reference, candidate)
    return score

In [3]:
from langchain.embeddings import OpenAIEmbeddings
import numpy as np

def cosine_similarity(vec1, vec2):
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return similarity

def sigmoid_scaled(similarity: float, threshold: float = 0.85, steepness: float = 50.0) -> float:
    """
    Преобразует similarity через сигмоид:
      - threshold — центр «перехода» (где output ≈ 0.5)
      - steepness — крутизна S-образной кривой
    """
    return 1.0 / (1.0 + np.exp(-steepness * (similarity - threshold)))


def compare_strings_llm(str1, str2):
    """
    Compare two strings using OpenAI embeddings and return cosine similarity.
    """
    embeddings = OpenAIEmbeddings()
    vec1 = embeddings.embed_query(str1)
    vec2 = embeddings.embed_query(str2)
    
    return sigmoid_scaled(cosine_similarity(vec1, vec2))

In [4]:
compare_strings_llm(
    "This code uses `tx.origin` for authorization checks. If an authorized account calls a malicious contract that interacts with this contract, the malicious contract can pass the authorization due to `tx.origin` returning the original sender's address. This can lead to unauthorized access and control over the contract.",
    "Directly updating the user's credit before checking the balance could lead to race conditions in certain scenarios, depending on how the `withdraw` function is used. If a malicious actor can call `deposit` right after their balance is checked in the `withdraw` function, they could exploit this to withdraw more than their deposit."
    )

C:\Users\bowsw\AppData\Local\Temp\ipykernel_10336\2134438155.py:21: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


0.09294196605612257

In [5]:
compare_strings_llm(
    "Using call to withdraw funds exposes the contract to reentrancy attacks, allowing malicious contracts to exploit the inconsistent state during the withdrawal process.",
    "The withdrawal of funds is done using `call` method after checking the user's credit. If a reentrant call is made, it could allow a malicious contract to withdraw more funds than intended before the state is updated (credit[msg.sender] is decremented). This could lead to arbitrary ETH losses."
    )

0.9718318044356793

In [6]:
hits = 0
count = 0

BLUE_MODE = False
LLM_MODE = True


for i in range(len(df)):
    vulnerabilities = df.iloc[i]["vulnerabilities"]
    contain = df.iloc[i]["contain"]
    miss = df.iloc[i]["miss"]

    if vulnerabilities == "nan":
        print("vulnerabilities is nan")
        continue

    # Filter vulnerabilities with high or medium severity
    vulnerabilities = json.loads(vulnerabilities)
    vulnerabilities = [v for v in vulnerabilities if v["severity"] in ["high", "medium"]]

    problems = [v["problem"] for v in vulnerabilities]
    lines = [v["line"] for v in vulnerabilities]
    tokens = [v["token"] for v in vulnerabilities]
    explanations = [v["explanation"] for v in vulnerabilities]
    migrations = [v["migration"] for v in vulnerabilities]

    
    cmp_func = compare_strings if BLUE_MODE else None
    cmp_func = compare_strings_llm if LLM_MODE else cmp_func

    # Process missed vulnerabilities
    for entry in json.loads(miss):
        if cmp_func:
            count += 2
            max_explanation_score = max( cmp_func(entry["explanation"], exp) for exp in explanations ) if explanations else 0
            max_migration_score = max( cmp_func(entry["migration"], mig) for mig in migrations ) if migrations else 0
            hits += 2 - max_migration_score - max_explanation_score

        
        count += 3
        hits += 1 if entry["line"] not in lines else 0
        hits += 1 if entry["problem"] not in problems else 0
        hits += 1 if entry["token"] not in tokens else 0

    # Process contained vulnerabilities
    for entry in json.loads(contain):
        if cmp_func:
            count += 2
            max_explanation_score = max( cmp_func(entry["explanation"], exp) for exp in explanations ) if explanations else 0
            max_migration_score = max( cmp_func(entry["migration"], mig) for mig in migrations ) if migrations else 0
            hits += max_migration_score + max_explanation_score

        count += 3
        hits += 1 if entry["line"] in lines else 0
        hits += 1 if entry["problem"] in problems else 0
        hits += 1 if entry["token"] in tokens else 0

accuracy = hits / count if count > 0 else 0
print("accuracy:", accuracy)


accuracy: 0.6205697813850948


In [7]:
for i in df["vulnerabilities"]:
    print(i)

[{"line": "10", "token": "require(payable(msg.sender).call{value: amount}(\"\"))", "problem": "Reentrancy Vulnerability", "severity": "high", "explanation": "The contract makes an external call to transfer ETH before updating the internal state (credit[msg.sender]), which can be exploited through reentrancy attacks by malicious contracts. An attacker can drain funds repeatedly before the credit gets updated.", "migration": "Use the Checks-Effects-Interactions pattern: update the user's credit balance before making the external call. Alternatively, consider using `transfer` instead of `call` for simpler fund transfers."}, {"line": "10", "token": "require(payable(msg.sender).call{value: amount}(\"\"))", "problem": "Unchecked External Call", "severity": "medium", "explanation": "The external call may fail due to various reasons (e.g., the recipient is a contract that does not accept ETH or reverts). If it fails, the contract will not revert, resulting in inconsistent state where the withd